# <u><span style="background-color: red;">__01_Fuentes__</span></u>  

Objetivos:
- Carga de datos orgininales (raw).
- Guardado de copias en processed (sin cambios).

Índice:
- [df_saleshourly_raw](#df_saleshourly_raw)
- [df_salesdaily_raw](#df_salesdaily_raw)
- [df_salesweekly_raw](#df_salesweekly_raw)
- [df_salesmonthly_raw](#df_salesmonthly_raw)

El objetivo de este notebook es dejar los datos **listos para el procesado** en la siguiente etapa.  
Aquí se ha hecho:  
- Lectura desde la fuente original (archivos CSV descargados de Kaggle).  
- Exploración inicial con `.head()`, `.info()`, `.describe()`.  
- Creación de una copia de trabajo (`*_processed`).  
- Guardado de los datasets en la carpeta `../data/processed/`.  

El notebook cierra entregando los datos en un estado **intermedio o staging**, preparados para el paso de limpieza y transformación que se abordará en el Notebook 02_LimpezaEDA.

En este notebook **no procede realizar una unión (`merge`)** de los cuatro datasets (`hourly`, `daily`, `weekly`, `monthly`).  
La razón es que cada uno representa una **granularidad distinta del mismo fenómeno (ventas)**.  
- *Hourly*: ventas por hora.  
- *Daily*: ventas agregadas por día.  
- *Weekly*: ventas agregadas por semana.  
- *Monthly*: ventas agregadas por mes.  

Unirlos directamente generaría duplicidades o inconsistencias, ya que las observaciones no son equivalentes.  
Por ello, se mantendrán **separados** y se trabajará con cada granularidad de forma independiente en fases posteriores.

---


## ¿Debe quedar `processed` vacío aquí?

Idealmente, en este notebook **no deberíamos llenar aún la carpeta `processed/`**.  
Lo correcto sería:  
- Dejar únicamente los datos originales en `raw/`.  
- Generar el contenido de `processed/` **en el Notebook 02 (Limpieza y Procesado)**, donde efectivamente se aplican transformaciones y se obtiene una versión lista para el modelado.  

De este modo, la separación entre fases queda más clara:  
- **01_Fuentes** → Descarga y staging (sólo `raw`).  
- **02_Procesado** → Limpieza y guardado de `processed`.  


Los grupos terapéuticos ATC que aparecen son:  

#### M – Sistema musculoesquelético (antiinflamatorios y antirreumáticos)
- `M01AB` – Antiinflamatorios no esteroideos (AINEs), derivados del acético y sustancias relacionadas.  
Ejemplos: diclofenaco, aceclofenaco, indometacina.  
Uso: tratamiento del dolor e inflamación en enfermedades musculoesqueléticas.

- `M01AE` – Antiinflamatorios no esteroideos (AINEs), derivados del propiónico.  
Ejemplos: ibuprofeno, naproxeno, ketoprofeno.  
Uso: dolor, inflamación y fiebre.

#### N – Sistema nervioso
- `N02BA` – Analgésicos, derivados del ácido salicílico.  
Ejemplo: ácido acetilsalicílico (aspirina).  
Uso: analgesia leve-moderada y propiedades antiagregantes en dosis bajas.

- `N02BE` – Otros analgésicos y antipiréticos.  
Ejemplos: paracetamol, metamizol.  
Uso: dolor leve-moderado y fiebre.

- `N05B` – Ansiolíticos  
Ejemplos: benzodiacepinas como diazepam, lorazepam.  
Uso: ansiedad, insomnio a corto plazo, tensión nerviosa.

- `N05C` – Hipnóticos y sedantes.  
Ejemplos: zolpidem, zopiclona, barbitúricos.  
Uso: insomnio y otros trastornos del sueño.

#### R – Sistema respiratorio
- `R03` – Medicamentos para enfermedades obstructivas de las vías respiratorias.  
Ejemplos: salbutamol, budesonida, montelukast.  
Uso: asma, EPOC y otras enfermedades respiratorias con broncoespasmo.

- `R06` – Antihistamínicos de uso sistémico.  
Ejemplos: loratadina, cetirizina, fexofenadina.  
Uso: alergias (rinitis, urticaria, prurito).

La página web donde extraigo los datos es Kaggle:

https://www.kaggle.com/datasets/milanzdravkovic/pharma-sales-data

In [1]:
# Importamos librerías
import pandas as pd

`os`: sirve para trabajar con rutas de forma cómoda. Por ejemplo, leer datos de /data/raw.  
La dejo importada por si tuviera que usarla al cargar los datasets.

Más adelante, agregaremos scikit-learn y otras librerías específicas para series temporales.

Cargo los 4 datasets que tengo.

In [2]:
# Cargar los datasets
# https://www.kaggle.com/datasets/milanzdravkovic/pharma-sales-data
df_saleshourly_raw = pd.read_csv("../data/raw/saleshourly.csv")
df_salesdaily_raw = pd.read_csv("../data/raw/salesdaily.csv")
df_salesweekly_raw = pd.read_csv("../data/raw/salesweekly.csv")
df_salesmonthly_raw = pd.read_csv("../data/raw/salesmonthly.csv")

Hago una vista rápida de los datasets usando info y describe.

## df_saleshourly_raw

In [3]:
df_saleshourly_raw.head()

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,1/2/2014 8:00,0.0,0.67,0.4,2.0,0.0,0.0,0.0,1.0,2014,1,8,Thursday
1,1/2/2014 9:00,0.0,0.00,1.0,0.0,2.0,0.0,0.0,0.0,2014,1,9,Thursday
2,1/2/2014 10:00,0.0,0.00,0.0,3.0,2.0,0.0,0.0,0.0,2014,1,10,Thursday
3,1/2/2014 11:00,0.0,0.00,0.0,2.0,1.0,0.0,0.0,0.0,2014,1,11,Thursday
4,1/2/2014 12:00,0.0,2.00,0.0,5.0,2.0,0.0,0.0,0.0,2014,1,12,Thursday


In [4]:
df_saleshourly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50532 entries, 0 to 50531
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datum         50532 non-null  object 
 1   M01AB         50532 non-null  float64
 2   M01AE         50532 non-null  float64
 3   N02BA         50532 non-null  float64
 4   N02BE         50532 non-null  float64
 5   N05B          50532 non-null  float64
 6   N05C          50532 non-null  float64
 7   R03           50532 non-null  float64
 8   R06           50532 non-null  float64
 9   Year          50532 non-null  int64  
 10  Month         50532 non-null  int64  
 11  Hour          50532 non-null  int64  
 12  Weekday Name  50532 non-null  object 
dtypes: float64(8), int64(3), object(2)
memory usage: 5.0+ MB


In [5]:
df_saleshourly_raw.describe()

,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour
count,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000,50532.000000
mean,0.209787,0.162365,0.161723,1.246842,0.368989,0.024736,0.229732,0.120870,2016.401409,6.344811,11.500475
std,0.556003,0.416109,0.453211,2.387392,0.930934,0.217871,1.240513,0.391999,1.664444,3.385761,6.921706
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2015.000000,3.000000,6.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016.000000,6.000000,12.000000
75%,0.000000,0.000000,0.000000,1.875000,0.000000,0.000000,0.000000,0.000000,2018.000000,9.000000,17.000000
max,7.000000,6.000000,6.500000,29.000000,15.000000,6.000000,25.000000,5.000000,2019.000000,12.000000,23.000000


## df_salesdaily_raw

In [6]:
df_salesdaily_raw.head()

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,1/2/2014,0.0,3.67,3.4,32.40,7.0,0.0,0.0,2.0,2014,1,248,Thursday
1,1/3/2014,8.0,4.00,4.4,50.60,16.0,0.0,20.0,4.0,2014,1,276,Friday
2,1/4/2014,2.0,1.00,6.5,61.85,10.0,0.0,9.0,1.0,2014,1,276,Saturday
3,1/5/2014,4.0,3.00,7.0,41.10,8.0,0.0,3.0,0.0,2014,1,276,Sunday
4,1/6/2014,5.0,1.00,4.5,21.70,16.0,2.0,6.0,2.0,2014,1,276,Monday


In [7]:
df_salesdaily_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datum         2106 non-null   object 
 1   M01AB         2106 non-null   float64
 2   M01AE         2106 non-null   float64
 3   N02BA         2106 non-null   float64
 4   N02BE         2106 non-null   float64
 5   N05B          2106 non-null   float64
 6   N05C          2106 non-null   float64
 7   R03           2106 non-null   float64
 8   R06           2106 non-null   float64
 9   Year          2106 non-null   int64  
 10  Month         2106 non-null   int64  
 11  Hour          2106 non-null   int64  
 12  Weekday Name  2106 non-null   object 
dtypes: float64(8), int64(3), object(2)
memory usage: 214.0+ KB


In [8]:
df_salesdaily_raw.describe()

,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour
count,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000
mean,5.033683,3.895830,3.880441,29.917095,8.853627,0.593522,5.512262,2.900198,2016.401235,6.344255,275.945869
std,2.737579,2.133337,2.384010,15.590966,5.605605,1.092988,6.428736,2.415816,1.665060,3.386954,1.970547
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000,1.000000,190.000000
25%,3.000000,2.340000,2.000000,19.000000,5.000000,0.000000,1.000000,1.000000,2015.000000,3.000000,276.000000
50%,4.990000,3.670000,3.500000,26.900000,8.000000,0.000000,4.000000,2.000000,2016.000000,6.000000,276.000000
75%,6.670000,5.138000,5.200000,38.300000,12.000000,1.000000,8.000000,4.000000,2018.000000,9.000000,276.000000
max,17.340000,14.463000,16.000000,161.000000,54.833333,9.000000,45.000000,15.000000,2019.000000,12.000000,276.000000


## df_salesweekly_raw

In [9]:
df_salesweekly_raw.head()

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06
0,1/5/2014,14.00,11.67,21.3,185.95,41.0,0.0,32.0,7.0
1,1/12/2014,29.33,12.68,37.9,190.70,88.0,5.0,21.0,7.2
2,1/19/2014,30.67,26.34,45.9,218.40,80.0,8.0,29.0,12.0
3,1/26/2014,34.00,32.37,31.5,179.60,80.0,8.0,23.0,10.0
4,2/2/2014,31.02,23.35,20.7,159.88,84.0,12.0,29.0,12.0


In [10]:
df_salesweekly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   datum   302 non-null    object 
 1   M01AB   302 non-null    float64
 2   M01AE   302 non-null    float64
 3   N02BA   302 non-null    float64
 4   N02BE   302 non-null    float64
 5   N05B    302 non-null    float64
 6   N05C    302 non-null    float64
 7   R03     302 non-null    float64
 8   R06     302 non-null    float64
dtypes: float64(8), object(1)
memory usage: 21.4+ KB


In [11]:
df_salesweekly_raw.describe()

,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,35.102441,27.167611,27.060295,208.627161,61.740853,4.138935,38.439811,20.224561
std,8.617106,7.043491,8.086458,76.069221,22.436970,3.129265,22.900873,11.381464
min,7.670000,6.237000,3.500000,86.250000,18.000000,0.000000,2.000000,1.000000
25%,29.387500,22.387500,21.300000,149.300000,47.000000,2.000000,21.000000,11.475000
50%,34.565000,26.789500,26.500000,198.300000,57.000000,3.979167,35.000000,17.500000
75%,40.175000,31.046500,32.475000,252.471500,71.000000,6.000000,51.000000,26.000000
max,65.330000,53.571000,60.125000,546.899000,154.000000,17.000000,131.000000,65.000000


## df_salesmonthly_raw

In [12]:
df_salesmonthly_raw.head()

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06
0,2014-01-31,127.69,99.090,152.100,878.030,354.0,50.0,112.0,48.2
1,2014-02-28,133.32,126.050,177.000,1001.900,347.0,31.0,122.0,36.2
2,2014-03-31,137.44,92.950,147.655,779.275,232.0,20.0,112.0,85.4
3,2014-04-30,113.10,89.475,130.900,698.500,209.0,18.0,97.0,73.7
4,2014-05-31,101.79,119.933,132.100,628.780,270.0,23.0,107.0,123.7


In [13]:
df_salesmonthly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   datum   70 non-null     object 
 1   M01AB   70 non-null     float64
 2   M01AE   70 non-null     float64
 3   N02BA   70 non-null     float64
 4   N02BE   70 non-null     float64
 5   N05B    70 non-null     float64
 6   N05C    70 non-null     float64
 7   R03     70 non-null     float64
 8   R06     70 non-null     float64
dtypes: float64(8), object(1)
memory usage: 5.1+ KB


In [14]:
df_salesmonthly_raw.describe()

,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,149.992000,116.514286,115.020843,892.542071,262.118571,17.842857,167.675000,86.662571
std,31.485325,27.889336,31.245899,338.843908,85.060930,8.481242,81.767979,45.859336
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,137.490000,103.518250,94.375000,648.187500,223.750000,12.000000,112.000000,49.875000
50%,154.635000,114.840000,117.225000,865.824500,250.300000,18.000000,160.000000,74.100000
75%,169.000000,128.359750,133.837500,1061.580000,293.650000,23.000000,218.250000,119.807500
max,211.130000,222.351000,191.600000,1856.815000,492.000000,50.000000,386.000000,213.040000


Usaré MAPE como métrica principal porque negocio entiende mejor los errores si se lo damos en %.  
Puedo complementar con MAE para ver el error en unidades absolutas.

In [15]:
print("Datasets RAW cargados y listos para el procesado en el Notebook 02_LimpiezaEDA.")

Datasets RAW cargados y listos para el procesado en el Notebook 02_LimpiezaEDA.
